# Parsing TFProf Files

TFProf files, generated for instance by [benchmark scripts](https://github.com/tensorflow/benchmarks/blob/master/scripts/tf_cnn_benchmarks/benchmark_cnn.py#L196), contain memory and time stats for every operation, recorded over one or several runs.

I use this notebook to select the operations I'm interested in, compute mean values over all test runs, and convert them to CSV, which is then exported to a spreadsheet app.

In [ ]:
from tensorflow.python.pywrap_tensorflow import ProfilerFromFile
from tensorflow.python.profiler import model_analyzer
from tensorflow.python import pywrap_tensorflow as pwtf
from tensorflow import gfile

PROFILE_PATH = './resnet50_v2_bnr_b64_log'
TMP_FILE_PATH = '/tmp/tfprof_tmp_output'

ProfilerFromFile(PROFILE_PATH.encode('utf-8'))

First, I extract textual information for each run (`step`), filtering the operations by name with regexes and extracting memory (`bytes`) and time (`micros`, `accelerator_micros`, `cpu_micros`) stats. There are additional [options](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/profiler/g3doc/options.md) available, which may come in handy in more complex scenarios.



In [ ]:
def profile_lines(step = 0):
  options = {
    'view': 'scope',
    'select': ['bytes', 'micros', 'accelerator_micros', 'cpu_micros'], #, 'float_ops'],
    'order_by': 'occurrence',
    'max_depth': 50,
    'min_bytes': 0,
    'account_type_regexes': ['.*'],
    'start_name_regexes': ['.*'],
    'show_name_regexes': [
      'v/tower_0/cg/(strided_slice|spatial_mean|Relu).+',
      'v/tower_0/cg/.*(FusedBatchNorm|Conv2D|MaxPool|MatMul|Relu|add)$',
      'v/tower_0/xentropy/sparse_softmax_cross_entropy_loss',
      'v/tower_0/gradients/.*(StridedSliceGrad|FusedBatchNormGrad|spatial_mean\d+_grad|Conv2DBackpropFilter|Conv2DBackpropInput|MaxPoolGrad|ReluGrad|BiasAddGrad)$'
    ],
    'output': 'file:outfile=' + TMP_FILE_PATH
  }

  options['step'] = step

  opts = model_analyzer._build_options(options)
  pwtf.Profile(options['view'].encode('utf-8'), opts.SerializeToString())

  with gfile.GFile(TMP_FILE_PATH, 'rb') as profile:
    profile_str = profile.read()

  return profile_str.decode('utf-8').split("\n")[15:]

lines_by_steps = [profile_lines(s) for s in range(10)]

Each line in the output contains one operation, e.g.:
```
'  v/tower_0/cg/resnet_v20/conv2/batchnorm2/FusedBatchNorm (51.38MB/51.38MB, 95us/95us, 0us/0us, 95us/95us)'
'  v/tower_0/gradients/v/tower_0/cg/resnet_v20/conv1/conv2d/Conv2D_grad/Conv2DBackpropFilter (199.68KB/199.68KB, 32us/32us, 0us/0us, 32us/32us)'
```

Stats are placed in the order I've specified above (`bytes, micros, accelerator_micros, cpu_micros`). The units, however, differ between operations (`MB` vs `KB` vs `B`), complicating further analysis. I convert them to a common representation (`KB` for memory usage and `us` for time elapsed):

In [ ]:
import numpy as np
import re

def convert_units(val):
  if val.endswith('KB'):
    return float(val[:-2])
  elif val.endswith("MB"):
    return float(val[:-2]) * 1024
  elif val.endswith("B"):
    return float(val[:-1]) / 1024
  elif val.endswith("us"):
    return float(val[:-2])
  elif val.endswith("ms"):
    return float(val[:-2]) * 1000

def parse_line(line):
  data = line.strip().split(" (")
  
  name = data[0]\
    .replace('v/tower_0/cg/', '')\
    .replace('v/tower_0/gradients/', '')\
    .replace('v/tower_0/xentropy/', '')\
    .replace('conv2d/', '')\
    .replace('Conv2D_grad/', '')\
    .replace('Relu_grad/', '')\
    .replace('FusedBatchNorm_grad/', '')
  
  name = re.sub(r'/batchnorm\d+/', '/', name)
  
  name_type = name.rsplit('/', 1)
  if len(name_type) == 1:
    if name.startswith('Relu_'): name_type.append('Relu')
    else: name_type.append('')
  
  return (name_type +
    [convert_units(stat.split('/')[-1]) for stat in data[1].replace(')', '').split(', ')])

parsed_lines_by_steps = np.array([[parse_line(l) for l in lines if l != ''] for lines in lines_by_steps])
print(f'Parsed {parsed_lines_by_steps.shape} profile entries; the shape is (runs, ops per run, stats per op)')

Finally, I compute the mean and the coefficient of variation over test runs and prepare the CSV output. The columns are `short node name, operation name, memory mean, memory cv, total time mean, total time cv, accelerator time mean, accelerator time cv, cpu time mean, cpu time cv`:

In [ ]:
import sys

mean = np.mean(parsed_lines_by_steps[:, :, 2:].astype('float64'), axis=0)
std = np.std(parsed_lines_by_steps[:, :, 2:].astype('float64'), axis=0)
cf_of_variation = np.nan_to_num(std / mean) * 100.

output_shape = parsed_lines_by_steps.shape[1:2] + (2 + 2 * mean.shape[1],)
output = np.empty(output_shape, dtype=parsed_lines_by_steps.dtype)

output[:, 0:2] = parsed_lines_by_steps[0, :, 0:2]
output[:, 2::2] = mean
output[:, 3::2] = cf_of_variation

np.savetxt(sys.stdout, output, fmt='%s', delimiter=',')